In [0]:
%sql
CREATE OR REPLACE TABLE workspace.ecommerce.ecom_bronze_events AS
SELECT
    CAST(data.user_id        AS STRING)   AS user_id,
    CAST(data.user_session   AS STRING)   AS session_id,
    CAST(data.event_time     AS TIMESTAMP)AS event_time,
    CAST(data.event_type     AS STRING)   AS event_type,
    CAST(data.product_id     AS BIGINT)   AS product_id,
    CAST(data.brand          AS STRING)   AS brand,
    CAST(data.category_code  AS STRING)   AS category,
    CAST(data.price          AS DOUBLE)   AS price,
    TO_DATE(data.event_time)              AS event_date
FROM read_files(
    '/Volumes/workspace/ecommerce/ecommerce_data/Batch/',
    format => 'csv',
    header => 'true'
) AS data;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.ecommerce.ecom_silver_events AS
SELECT *
FROM (
    SELECT *, row_number() OVER (PARTITION BY user_id, session_id, event_time, event_type ORDER BY product_id) AS rn
    FROM workspace.ecommerce.ecom_bronze_events
    WHERE event_type IN ('view', 'cart', 'purchased')
      AND user_id IS NOT NULL
      AND session_id IS NOT NULL
      AND product_id IS NOT NULL
) sub
WHERE rn = 1;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.ecommerce.ecom_gold_user_viewed AS
WITH viewed AS (
  SELECT
    user_id, product_id, brand, category, price, event_time,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY event_time DESC) AS rn
  FROM ecommerce.ecom_silver_events
  WHERE event_type = 'view'
)
SELECT user_id, product_id, brand, category, price, event_time
FROM viewed
WHERE rn <= 20;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.ecommerce.ecom_gold_user_cart AS
  SELECT
    user_id, product_id, brand, category, price, event_time
  FROM ecommerce.ecom_silver_events
  WHERE event_type = 'cart'
  ORDER BY event_time DESC

